In [ ]:
import os
import sys
import asyncio
import json
from typing import Any, Dict, List
from pathlib import Path
# Add the src directory to Python path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from subagent.eda_agent import EDAAgent
from examples.agent_app import App
from agentdk.core.logging_config import ensure_nest_asyncio


def get_llm():    
    # Try OpenAI
    if os.getenv('OPENAI_API_KEY'):
        try:
            from langchain_openai import ChatOpenAI
            model = "gpt-4o-mini"
            llm = ChatOpenAI(model=model, temperature=0)
            print(f"✅ Using OpenAI {model}")
            return llm
        except ImportError:
            print("❌ langchain_openai not available")
        except Exception as e:
            print(f"❌ OpenAI setup failed: {e}")
    
    # Try Anthropic
    elif os.getenv('ANTHROPIC_API_KEY'):
        try:
            from langchain_anthropic import ChatAnthropic
            llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
            print("✅ Using Anthropic Claude-3-Haiku")
            return llm
        except ImportError:
            print("❌ langchain_anthropic not available")
        except Exception as e:
            print(f"❌ Anthropic setup failed: {e}")

    else:
        raise ValueError("No LLM API key found")
    # Fallback to mock
    return llm

# Get LLM instance
llm = get_llm()



✅ Environment variables loaded from /home/zineng/workspace/agentic/agentdk/examples/.env
✅ Using OpenAI gpt-4o-mini


In [2]:
agent = App(model=llm, memory=True)
print(agent("which table i have access to?"))
print(agent("what is the total transaction amount from customer 'John Smith'"))

2025-06-25 11:42:38,683 - agentdk - INFO - WorkingMemory initialized for user default, session 9d93ef3e-1eef-4b15-b0d0-3b490f6acc44
2025-06-25 11:42:38,686 - agentdk.memory.episodic_memory - INFO - EpisodicMemory initialized for user default
2025-06-25 11:42:38,687 - agentdk - INFO - FactualMemory initialized for user default
2025-06-25 11:42:38,688 - agentdk - INFO - Initializing WorkingMemory
2025-06-25 11:42:38,688 - agentdk - INFO - WorkingMemory initialization complete
2025-06-25 11:42:38,689 - agentdk - INFO - Initializing FactualMemory database
2025-06-25 11:42:38,690 - agentdk - INFO - FactualMemory initialization complete
2025-06-25 11:42:38,691 - agentdk.memory.memory_manager - INFO - MemoryManager initialized for user default, session session_2e3a0b5e
✅ Memory system initialized for user default
2025-06-25 11:42:39,562 - agentdk - INFO - Loaded MCP configuration from: subagent/mcp_config.json
2025-06-25 11:42:39,702 - agentdk - INFO - MCP client configured with 1 servers
202

In [3]:
print(agent("give me 2 records from table 'trans'"))

2025-06-25 11:42:53,273 - agentdk - INFO - {"tool": "execute_sql", "args": {"query": "SELECT * FROM transactions LIMIT 2"}}
2025-06-25 11:42:53,677 - agentdk - INFO - {"tool": "execute_sql", "status": "completed"}
```sql
SELECT * FROM transactions LIMIT 2
```

**Result:**

| transaction_id | account_id | transaction_type | amount  | currency | transaction_date     | description        | reference_number       | status    | merchant_name | category |
|----------------|------------|-------------------|---------|----------|-----------------------|---------------------|------------------------|-----------|---------------|----------|
| 1              | 1          | deposit            | 1200.00 | USD      | 2024-01-15 09:30:00   | Payroll deposit      | PAY-20240115-001       | completed | None          | salary   |
| 2              | 1          | withdrawal         | 80.00   | USD      | 2024-01-16 14:25:00   | ATM withdrawal       | ATM-20240116-001       | completed | None          | cash

In [4]:
print(agent("which table i just accessed'"))

The table you just accessed is **transactions**.
